In [1]:
# import os
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…

In [11]:
# Define the path to the shapefile
shp_path = "E:/sa_satellite_files/eastern_cape/shape_files/eastern_cape.shp"

# Convert the shapefile to an Earth Engine geometry
roi = geemap.shp_to_ee(shp_path).geometry()

# Load Sentinel-2 data
collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi) \
    .filterDate('2021-06-01', '2021-08-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .filterBounds(roi)

# Select the desired bands
bands = ['B4', 'B3', 'B2']  # Example: Selecting bands 2, 3, and 4

# Apply median filter
image = collection.select(bands).median()

# Define the CRS and scale
crs = image.projection().crs()
scale = 1000

# Display the image on the map
vis_params = {'bands': bands, 'min': 0, 'max': 2500}  # Example: Set the min and max values
Map.addLayer(image.clip(roi), vis_params, 'Sentinel-2')

# Center the map on the ROI
Map.centerObject(roi, 6)

# Get the image footprint within the ROI
footprint = image.geometry().intersection(roi, ee.ErrorMargin(0.5))

# Download the image
geemap.download_ee_image(image, filename='my_sentinel_8', scale=scale, crs=crs, region=footprint)

my_sentinel_8: |          | 0.00/9.38M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


In [5]:
shp_path = "E:/sa_satellite_files/eastern_cape/shape_files/eastern_cape.shp"
roi = geemap.shp_to_ee(shp_path)

# Load Sentinel-2 data
collection = ee.ImageCollection('COPERNICUS/S2_SR')\
.filterBounds(roi).filterDate('2021-06-01', '2021-08-31')\
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
.filterBounds(roi)

# Select the desired bands
bands = ['B4', 'B3', 'B2']  # Example: Selecting bands 2, 3, and 4

# Apply median filter
image = collection.select(bands).median()

# Define the CRS, region, and scale
crs = image.projection().crs()
scale = 20
region = roi.geometry()

# Display the image on the map
vis_params = {'bands': bands, 'min': 0, 'max': 2500}  # Example: Set the min and max values
Map.addLayer(image.clip(roi), vis_params, 'Sentinel-2')

Map.centerObject(roi, 6)

# Download the image
geemap.download_ee_image(image, "my_sentinel_7.tif", scale=100, crs=crs, region=region)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\CJay\Documents\GitHub\virtual_football_league_predictor\my_sentinel_7.tif


In [16]:
# Download the image
geemap.download_ee_image(image, "my_sentinel_7.tif", scale=100, crs=crs, region=region)

# save_path = "C:/Users/CJay/Downloads/south_africa_shapes/eastern_cape/my_sentinel_2.tif"
# geemap.download_ee_image(image, filename=save_path, scale=scale, crs=crs, region=region)

my_sentinel_7.tif: |          | 0.00/9.38M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


In [ ]:

# features = geemap.fishnet(roi, rows=130, cols=200, delta=0.1)
features = geemap.fishnet(roi, rows=20, cols=30, delta=0)
Map.centerObject(roi)
Map.addLayer(features, {}, 'Grids')

In [ ]:
out_dir = "E:/sa_satellite_files/eastern_cape/image_tiles"

geemap.download_ee_image_tiles(
    image, features, out_dir, prefix="east_cape_", crs=crs, scale=500
)

In [ ]:
tif_image_directory = "C:/Users/CJay/Downloads/south_africa_shapes/eastern_cape/satellite_tiff_image.tif"
output_directory = "C:/Users/CJay/Downloads/south_africa_shapes/eastern_cape/all_images"


In [ ]:
# Define the location of interest
loc = ee.Geometry.Point(23.241, -30.64)

# Load the shapefile and convert it to an Earth Engine object
shp_path = "C:/Users/CJay/Downloads/sa_satellite_files/eastern_cape/shape_files/eastern_cape.shp"
roi = geemap.shp_to_ee(shp_path)

# Load the nightlight image collection
nightlights = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG').filterDate('2021-06-01', '2021-08-31')

# Filter the image collection to the region of interest
nightlights_roi = nightlights.filterBounds(roi)

# Calculate the median of the nightlight intensity within the date range and region
median_nightlight = nightlights_roi.median()

# Get the median value at the location of interest
median_value = median_nightlight.reduceRegion(reducer=ee.Reducer.median(), geometry=loc, scale=20).get('avg_rad').getInfo()

# Print the median value
print('Median Nightlight Intensity:', median_value)

# Create a folium map
Map = geemap.Map()

# Add the shapefile to the map
Map.addLayer(roi, {}, 'Eastern Cape')

# Add the nightlight image as a tile layer to the map (selecting a single band)
Map.addLayer(median_nightlight.select('avg_rad'), {'min': 0, 'max': 100}, 'Nightlight Intensity')

# Center the map on the location of interest
Map.centerObject(loc, 6)

# Display the map
Map

## Converting multiple satellite images to jpg

In [37]:
import numpy as np
import tifffile as tiff
from PIL import Image

large_image_stack = tiff.imread("E:/sa_satellite_files/eastern_cape/image_tiles/new_sentinel_6_23_50.tif")

# Convert the image stack to a single image
large_image = np.concatenate(large_image_stack, axis=0)

# Normalize the image values to 0-255 range
large_image = ((large_image - large_image.min()) / (large_image.max() - large_image.min())) * 255

# Convert the image to uint8 data type
large_image = large_image.astype(np.uint8)

# Convert the image to PIL Image
pil_image = Image.fromarray(large_image)

# Convert to RGB color mode
rgb_image = pil_image.convert("RGB")

# Save the RGB image as PNG
save_path = "E:/sa_satellite_files/eastern_cape/converted_images/converted_image.png"
rgb_image.save(save_path)

print("Conversion complete.")

Conversion complete.


In [ ]:
from PIL import Image

image_path = 'E:/sa_satellite_files/eastern_cape/image_tiles/new_sentinel_6_01_01.tif'

try:
    # Open and verify the image file
    with Image.open(image_path) as image:
        image.verify()
        print(f"The image file '{image_path}' is valid.")
except (IOError, SyntaxError) as e:
    print(f"Error: Could not open the image file '{image_path}' - {e}")